In [9]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
import seaborn as sns

In [10]:
import os
import pandas as pd

def merge(df1: pd.DataFrame, df2: pd.DataFrame, merge_on: str, check_col: str) -> pd.DataFrame:
    """
    
    Merges df1 and df2

    Args:
        df1 (pd.DataFrame): First Excel Sheet
        df2 (pd.DataFrame): Second Excel Sheet
        merge_on (str): Column to merge on
        check_col (str): Colum to check merge

    Returns:
        pd.DataFrame: Merged Excel Sheet
    """
    
    df = pd.merge(left=df1, right= df2, how= "left", on= merge_on)
    
    if df[f"{check_col}_x"].equals(df[f"{check_col}_y"]):
        return df
    else:
        return ValueError(f"Values in the '{check_col}' column do not match after the merge.")

def standardize(df: pd.DataFrame, x: str):
    """
    
    This function will convert the indicator values from percents to values by\
        multiplying the percents to the country's MPI

    Args:
        df (pd.DataFrame): Original DataFrame
        x (str): Column Name
    """
    df.loc[:, x] = df["MPI"] * df[x] / 100

def get_region_dict(regions: list, df: pd.DataFrame) -> dict:
    """_summary_

    Args:
        regions (list): _description_
        df (pd.DataFrame): _description_

    Returns:
        dict: _description_
    """
    region_dict = {}
    for region in regions:
        region_df = df.loc[df["Region"] == region].drop(columns= "Region")
        region_dict[region] = region_df
    
    return region_dict

def get_weights(df: pd.DataFrame, pop_col: str) -> pd.DataFrame:
    """
    Returns DataFrame with population weight column

    Args:
        df (pd.DataFrame): Original DataFrame
        pop_col (str): Population Column Name

    Returns:
        pd.DataFrame: DataFrame column with weighted column
    """
    total_pop = df[pop_col].sum()
    df.loc[:, "Weight"] = df[pop_col] / total_pop
    return df

def get_weighted_cols(df: pd.DataFrame):
    """
    Creates weighted indicator columns

    Args:
        df (pd.DataFrame): DataFrame to transform
    """
    
    needs_weights = ['Nutrition', 'Child Mortality', 'Years of Schooling',
            'School Attendance', 'Cooking Fuel', 'Sanitation', 'Drinking Water',
            'Electricity', 'Housing', 'Assets']
    
    for col in needs_weights:
        col_name = col + "_w"
        df.loc[:, col_name] = df[col] * df["Weight"]
    

def get_hels(df: pd.DataFrame):
    """
    Creates normal and weighted Health, Education, and Living Standards columns

    Args:
        df (pd.DataFrame): DataFrame to transform
    """
    df.loc[:, "Health"] = df["Nutrition"] + df["Child Mortality"]
    df.loc[:, "Education"] = df["Years of Schooling"] + df["School Attendance"]
    df.loc[:, "Living Standards"] = (df["Cooking Fuel"] + df["Sanitation"] + df["Drinking Water"]
                                    + df["Electricity"] + df["Housing"] + df["Assets"])

    df.loc[:, "Health_w"] = df["Nutrition_w"] + df["Child Mortality_w"]
    df.loc[:, "Education_w"] = df["Years of Schooling_w"] + df["School Attendance_w"]
    df.loc[:, "Living Standards_w"] = (df["Cooking Fuel_w"] + df["Sanitation_w"] + df["Drinking Water_w"]
                                    + df["Electricity_w"] + df["Housing_w"] + df["Assets_w"])

def gather_dfs(year: int | list) -> dict[int, pd.DataFrame]:
    """
    Generates merged DataFrames for every year

    Args:
        year (int | list): Int or List of the year/s to gather
    
    Returns:
        dict[int, pd.DataFrame]: Dictionary containing all merged DataFrames, with years as keys
    """
    
    if isinstance(year, int):
        years = [year]
    elif isinstance(year, list):
        years = year
    
    dfs = {}
    checked = set()
    
    for year in years:
        if year not in checked:
            file_path = f"../../data/raw/Global MPI {year} National Results.xlsx"
            
            if os.path.exists(file_path):
                
                col1 = ["ISO Country Numeric Code", "ISO Country Code", "Country", "Region",
                        "Survey", "Survey Year", "MPI", "Headcount", "Intensity", "Vulnerable to Poverty",
                        "Severe Poverty", "Headcount Destitution", "MPI Poor Destitute", "Poor Variance",
                        "Population YOS", f"Population {year - 3}", f"Population {year - 2}", "Poor YOS",
                        f"Poor Population {year - 3}", f"Poor Population {year - 2}", "Total Indicators",
                        "Indicator Missing"]
                
                col2 = ["ISO Country Numeric Code", "ISO Country Code", "Country", "Region", "Survey", "Year",
                        "MPI", "Health", "Education", "Living Standards", "Nutrition", "Child Mortality", "Years of Schooling",
                        "School Attendance", "Cooking Fuel", "Sanitation", "Drinking Water", "Electricity", "Housing",
                        "Assets", "Total Indicators", "Missing Indicators"]
                
                df_1 = pd.read_excel(file_path,
                        sheet_name= "1.1 National MPI Results",
                        skiprows=8, skipfooter=10, names = col1)
                df_2 = pd.read_excel(file_path,
                                sheet_name= "1.3 Contribut'n of Deprivations",
                                skiprows=8, skipfooter=3, names= col2)
                
                df = merge(df_1, df_2, "ISO Country Code", "MPI") 
                df = df[["Country_x", 'MPI_x',
                    'Intensity', f'Population {year- 2}', "Region_y", 'Nutrition', 'Child Mortality',
                    'Years of Schooling', 'School Attendance', 'Cooking Fuel',
                    'Sanitation', 'Drinking Water', 'Electricity', 'Housing', 'Assets']]
                
                df = df.rename(columns={"Country_x": "Country", "MPI_x": "MPI", "Region_y": "Region"})
                
                indicator_list = ['Nutrition', 'Child Mortality',
                'Years of Schooling', 'School Attendance', 'Cooking Fuel',
                'Sanitation', 'Drinking Water', 'Electricity', 'Housing', 'Assets']
                
                for x in indicator_list:
                    standardize(df, x)
                    
                dfs[year] = df
                checked.add(year)
            
            else:
                raise ValueError(f"{file_path} file does not exist.")
            
    return dfs

def get_regionals(year: int | list, year_dict: dict):
    """
    Creates csv files of regional information

    Args:
        year (int | list): Int or List of the year/s to gather
        year_dict (dict): Dictionary containing all the DataFrames
    """
    if isinstance(year, int):
        years = [year]
    elif isinstance(year, list):
        years = year
    checked = set()
    
    for year in years:
        if year not in checked:
            
            full_df = year_dict[year]
            regions = list(full_df["Region"].unique())
            region_dict = get_region_dict(regions, full_df)
            region_dict["Global"] = full_df
                
            for region in region_dict.keys():
                var_name = region.lower().replace(" ", "_").replace("-", "_") + f"_{year}"
                region_df = get_weights(region_dict[region], f"Population {year - 2}")
                
                get_weighted_cols(region_df)
                get_hels(region_df)
                
                # output_filepath = f"../../data/interm/{var_name}.csv"
                # region_df.to_csv(output_filepath)


def main():
    YEARS = [2022, 2023]
    get_regionals(YEARS, gather_dfs(YEARS))

In [11]:
bbb = gather_dfs([2022, 2023])
bbb.keys()

dict_keys([2022, 2023])

In [12]:
isinstance([2022, 2023], list)

True

In [48]:
df = pd.read_csv("../../data/interm/global_2023.csv", index_col=0)
df_og = pd.read_excel("../../data/raw/Global MPI 2023 National Results.xlsx",sheet_name= "1.3 Contribut'n of Deprivations",
                      skiprows= 8, skipfooter= 3, names= ["ISO Country Numeric Code", "ISO Country Code", "Country", "Region", "Survey", "Year",
                        "MPI", "Health", "Education", "Living Standards", "Nutrition", "Child Mortality", "Years of Schooling",
                        "School Attendance", "Cooking Fuel", "Sanitation", "Drinking Water", "Electricity", "Housing",
                        "Assets", "Total Indicators", "Missing Indicators"])

In [49]:
df_og.loc[3]

ISO Country Numeric Code                        795
ISO Country Code                                TKM
Country                                Turkmenistan
Region                      Europe and Central Asia
Survey                                         MICS
Year                                           2019
MPI                                        0.000849
Health                                    82.404423
Education                                  15.49705
Living Standards                           2.098525
Nutrition                                  47.26482
Child Mortality                           35.139605
Years of Schooling                              0.0
School Attendance                          15.49705
Cooking Fuel                                    NaN
Sanitation                                      0.0
Drinking Water                                  0.0
Electricity                                     0.0
Housing                                    2.098524
Assets      

In [50]:
df[["Health", "Education", "Living Standards"]]

,Health,Education,Living Standards
0,0.000134,0.000174,0.000125
1,0.000228,0.000254,0.000208
2,0.000508,0.000238,0.000094
3,0.000700,0.000132,0.000018
4,0.000587,0.000296,0.000362
...,...,...,...
105,0.068782,0.121936,0.195210
106,0.097433,0.111026,0.200403
107,0.093184,0.128310,0.239853
108,0.098985,0.189093,0.228933


In [51]:
df[["Health", "Education", "Living Standards"]].sum(axis= 1) - df["MPI"]

0     -9.996344e-17
1     -2.000353e-16
2     -7.264155e-17
3     -1.056013e-16
4     -9.996344e-17
           ...     
105   -4.940492e-15
106    5.051515e-15
107   -6.772360e-15
108    3.275158e-14
109   -1.065814e-14
Length: 110, dtype: float64

In [52]:
df.loc[2, ["Health", "Education", "Living Standards"]].sum()

0.0008404317588392273

In [45]:
df.loc[2]

Country                                 Ukraine
MPI                                     0.00084
Intensity                             34.410454
Population 2021                       43531.422
Region                  Europe and Central Asia
Nutrition                                   NaN
Child Mortality                        0.000508
Years of Schooling                     0.000153
School Attendance                      0.000085
Cooking Fuel                           0.000037
Sanitation                             0.000016
Drinking Water                         0.000013
Electricity                            0.000002
Housing                                0.000008
Assets                                 0.000017
Weight                                 0.007086
Nutrition_w                                 NaN
Child Mortality_w                      0.000004
Years of Schooling_w                   0.000001
School Attendance_w                    0.000001
Cooking Fuel_w                          